In [2]:
%matplotlib inline
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [8]:
df = pd.read_csv('../data/grid_224_2/res.csv')
df_org = pd.read_csv('../data/input/train.csv')
df_pred = pd.read_csv('../data/output/pred_res.csv')

In [4]:
df.head()

,image_id,score_0,score_1,score_2,score_3,score_4,score_5
0,0018ae58b01bdadc8e347995b69f99aa_1_2,0.136081,0.804548,0.059371,0.0,0.0,0.0
1,0018ae58b01bdadc8e347995b69f99aa_2_2,0.000000,0.948043,0.051957,0.0,0.0,0.0
2,0018ae58b01bdadc8e347995b69f99aa_2_3,0.267339,0.722676,0.009985,0.0,0.0,0.0
3,0018ae58b01bdadc8e347995b69f99aa_3_2,0.000000,0.962552,0.037448,0.0,0.0,0.0
4,0018ae58b01bdadc8e347995b69f99aa_3_3,0.294882,0.696787,0.008331,0.0,0.0,0.0


In [5]:
df_pred.head()

,score_0,score_1,score_2,score_3,score_4,score_5,image_id
0,0.341262,0.555900,0.000256,0.000228,0.038028,0.064327,0005f7aaab2800f6170c399693a96917_10_7
1,0.097811,0.739216,0.000909,0.002047,0.123097,0.036921,0005f7aaab2800f6170c399693a96917_10_8
2,0.169699,0.659484,0.002739,0.002866,0.107763,0.057448,0005f7aaab2800f6170c399693a96917_11_8
3,0.444878,0.416824,0.001271,0.000805,0.052206,0.084016,0005f7aaab2800f6170c399693a96917_11_9
4,0.280059,0.528340,0.004801,0.003822,0.142662,0.040316,0005f7aaab2800f6170c399693a96917_12_8


In [9]:
df_org.head()

,image_id,data_provider,isup_grade,gleason_score
0,0005f7aaab2800f6170c399693a96917,karolinska,0,0+0
1,000920ad0b612851f8e01bcc880d9b3d,karolinska,0,0+0
2,0018ae58b01bdadc8e347995b69f99aa,radboud,4,4+4
3,001c62abd11fa4b57bf7a6c603a11bb9,karolinska,4,4+4
4,001d865e65ef5d2579c190a0e0350d8f,karolinska,0,0+0


In [47]:
score = '3+5'
sample = df_org[(df_org['gleason_score'] == score) & (df_org['data_provider'] == 'radboud')].sample(10)
print('Gleason Score: ', score)
for tar in sample['image_id'].values:

    _df = df[df['image_id'].str.contains(tar)].reset_index(drop=True)
    _df_pred = df_pred[df_pred['image_id'].str.contains(tar)].reset_index(drop=True)

    _df = pd.merge(_df, _df_pred, how='left', on='image_id', suffixes=['_org', '_pred'])
    temp = _df[_df.columns[1:]]
    _len = len(temp)
    temp = np.sum(temp, axis=0)
    print(tar)
    print(_len)
    print(temp)
    print('#'*30)

Gleason Score:  3+5
88fcb6de1ead418b238f198a635179c0
60
score_0_org     13.751036
score_1_org     40.503667
score_2_org      2.792690
score_3_org      2.132693
score_4_org      0.000000
score_5_org      0.819914
score_0_pred    14.499207
score_1_pred    40.520874
score_2_pred     2.290397
score_3_pred     0.949358
score_4_pred     1.286376
score_5_pred     0.453790
dtype: float64
##############################
a12ac7f1bc26fe59d642674fcbe45a30
52
score_0_org     15.398139
score_1_org     35.537867
score_2_org      0.525211
score_3_org      0.360750
score_4_org      0.000000
score_5_org      0.178033
score_0_pred    15.325197
score_1_pred    34.350506
score_2_pred     0.640361
score_3_pred     0.133693
score_4_pred     0.790979
score_5_pred     0.759264
dtype: float64
##############################
df8bd94723b96652f71fbdde64fc8669
77
score_0_org     12.691446
score_1_org     54.751076
score_2_org      2.760862
score_3_org      6.435547
score_4_org      0.000000
score_5_org      0.361069


In [44]:
score = '3+5'

sample = df_org[(df_org['gleason_score'] == score) & (df_org['data_provider'] == 'karolinska')].sample(10)
for tar in sample['image_id'].values:
    _df_pred = df_pred[df_pred['image_id'].str.contains(tar)].reset_index(drop=True)
    temp = _df_pred[_df_pred.columns[:-1]]
    temp = np.sum(temp, axis=0)
    print(tar)
    print(temp)
    print('#'*30)

05819281002c55258bb3086cc55e3b48
score_0     9.259028
score_1    17.797729
score_2     0.030472
score_3     0.117580
score_4     5.461469
score_5     4.333721
dtype: float64
##############################
fb2f9184b3ae3a323a44f83706ffce7d
score_0    12.998207
score_1    30.502782
score_2     0.546319
score_3     0.768333
score_4     3.176318
score_5     1.008042
dtype: float64
##############################
ebcc8512888bfadf6c90598a7e0b466b
score_0    14.742210
score_1    26.788935
score_2     0.108626
score_3     0.272274
score_4    11.591616
score_5     6.496339
dtype: float64
##############################
b9be346a547053539266d97872b5c7ab
score_0    12.754526
score_1    25.146285
score_2     0.089648
score_3     0.889670
score_4    12.690779
score_5     3.429092
dtype: float64
##############################
ea295f8383b74ad29cf5ba2daea89f98
score_0    14.332005
score_1    29.230968
score_2     0.111688
score_3     0.233347
score_4    13.799213
score_5    10.292779
dtype: float64
######